In [1]:
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import random
import matplotlib.pyplot as plt
%matplotlib inline
from image_helper import *
from parse_xml_annotations import *
from features import *
from reinforcement import *
from metrics import *
import logging
import time
import os


In [2]:
torch.__version__

'1.13.1'

In [3]:
torch.cuda.is_available()


True

# Load Image and Model

In [6]:
path_voc = "C:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2007"

# get models 
print("load models")

model_vgg = getVGG_16bn("../models")
model_vgg = model_vgg.cuda()
model = get_q_network()
model = model.cuda()

# define optimizers for each model
optimizer = optim.Adam(model.parameters(),lr=1e-6)
criterion = nn.MSELoss().cuda()   

# get image datas
print("load images")

path_voc = "C:/startcoding/Active-Object-Localization-Deep-Reinforcement-Learning-master/Active-Object-Localization-Deep-Reinforcement-Learning-master/datasets/VOCdevkit/VOC2007"
image_names = np.array(load_images_names_in_data_set('aeroplane_trainval', path_voc))
labels = load_images_labels_in_data_set('aeroplane_trainval', path_voc)
image_names_aero = []
for i in range(len(image_names)):
    if labels[i] == '1':
        image_names_aero.append(image_names[i])
image_names = image_names_aero
images = get_all_images(image_names, path_voc)

print("aeroplane_trainval image:%d" % len(image_names))

load models


d:\coding\Pytorch_Deep_RL_1-master\Pytorch_Deep_RL_1-master\reinforcement.py:82: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


load images
aeroplane_trainval image:238


### the replay part should be added in replay.py

In [7]:
from collections import namedtuple
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [13]:
epsilon = 0.9
BATCH_SIZE = 100
GAMMA = 0.90
CLASS_OBJECT = 1
steps = 10
epochs = 25
memory = ReplayMemory(1000)

def select_action(state):
    if random.random() < epsilon:
        action = np.random.randint(1,7)
    else:
        qval = model(Variable(state))
        _, predicted = torch.max(qval.data,1)
        action = predicted[0] + 1
    return action

def optimizer_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
    next_states = [s for s in batch.next_state if s is not None]
    non_final_next_states = Variable(torch.cat(next_states), 
                                     volatile=True).type(Tensor)
    state_batch = Variable(torch.cat(batch.state)).type(Tensor)
    action_batch = Variable(torch.LongTensor(batch.action).view(-1,1)).type(LongTensor)
    reward_batch = Variable(torch.FloatTensor(batch.reward).view(-1,1)).type(Tensor)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken => 모델에 현 상태를 넣고 내가 취한 액션의 reward를 가져옴
    state_action_values = model(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states. 다음상태의 action에 대한 reward들을 가져오고 가장 max한 action의 reward 선택
    next_state_values = Variable(torch.zeros(BATCH_SIZE, 1).type(Tensor)) 
    next_state_values[non_final_mask.view(-1,1)] = model(non_final_next_states).max(1)[0]
    
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    # 이제 우리는 휘발성 플래그로 손실을 엉망으로 만들고 싶지 않으므로 이를 지웁시다. 그 후에는 require_grad=False인 변수로 끝납니다.
    next_state_values.volatile = False
    
    # Compute the expected Q values 감마 곱해주고 리워드 더해주기
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute  loss
    loss = criterion(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Train the model

In [14]:
# train procedure
print('train the Q-network')
for epoch in range(epochs):
    print('epoch: %d' %epoch)
    now = time.time()
    for i in range(len(image_names)):
        # the image part
        image_name = image_names[i]
        image = images[i]
        annotation = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc)
        classes_gt_objects = get_ids_objects_from_annotation(annotation)
        gt_masks = generate_bounding_box_from_annotation(annotation, image.shape) 
         
        # the iou part
        original_shape = (image.shape[0], image.shape[1])
        region_mask = np.ones((image.shape[0], image.shape[1]))
        #choose the max bouding box
        iou = find_max_bounding_box(gt_masks, region_mask, classes_gt_objects, CLASS_OBJECT)
        
        # the initial part
        region_image = image
        size_mask = original_shape
        offset = (0, 0)
        history_vector = torch.zeros((4,6))
        state = get_state(region_image, history_vector, model_vgg)
        done = False

        for step in range(steps):

            # Select action, the author force terminal action if case actual IoU is higher than 0.5  실제 IoU가 0.5보다 큰 경우 작성자 강제 종료 조치
            if iou > 0.5:
                action = 6
            else:
                action = select_action(state)
            
            # Perform the action and observe new state
            if action == 6:
                next_state = None
                reward = get_reward_trigger(iou)
                done = True
            else:
                offset, region_image, size_mask, region_mask = get_crop_image_and_mask(original_shape, offset,
                                                                   region_image, size_mask, action)
                # update history vector and get next state
                history_vector = update_history_vector(history_vector, action)
                next_state = get_state(region_image, history_vector, model_vgg)
                
                # find the max bounding box in the region image
                new_iou = find_max_bounding_box(gt_masks, region_mask, classes_gt_objects, CLASS_OBJECT)
                reward = get_reward_movement(iou, new_iou)
                iou = new_iou
            print('epoch: %d, image: %d, step: %d, reward: %d' %(epoch ,i, step, reward))    
            # Store the transition in memory
            memory.push(state, action-1, next_state, reward)
            
            # Move to the next state
            state = next_state
            
            # Perform one step of the optimization (on the target network)
            optimizer_model()
            if done:
                break
    if epsilon > 0.1:
        epsilon -= 0.1
    time_cost = time.time() - now
    print('epoch = %d, time_cost = %.4f' %(epoch, time_cost))


train the Q-network
epoch: 0
epoch: 0, image: 0, step: 0, reward: 1
epoch: 0, image: 0, step: 1, reward: 1
epoch: 0, image: 0, step: 2, reward: 3
epoch: 0, image: 1, step: 0, reward: 1
epoch: 0, image: 1, step: 1, reward: 1
epoch: 0, image: 1, step: 2, reward: -1
epoch: 0, image: 1, step: 3, reward: -1
epoch: 0, image: 1, step: 4, reward: -3
epoch: 0, image: 2, step: 0, reward: 3
epoch: 0, image: 3, step: 0, reward: 1
epoch: 0, image: 3, step: 1, reward: -3
epoch: 0, image: 4, step: 0, reward: 1
epoch: 0, image: 4, step: 1, reward: -1
epoch: 0, image: 4, step: 2, reward: -1
epoch: 0, image: 4, step: 3, reward: -1
epoch: 0, image: 4, step: 4, reward: -1
epoch: 0, image: 4, step: 5, reward: -1
epoch: 0, image: 4, step: 6, reward: -1
epoch: 0, image: 4, step: 7, reward: -1
epoch: 0, image: 4, step: 8, reward: -1
epoch: 0, image: 4, step: 9, reward: -1
epoch: 0, image: 5, step: 0, reward: -1
epoch: 0, image: 5, step: 1, reward: -1
epoch: 0, image: 5, step: 2, reward: -1
epoch: 0, image: 5,

C:\Users\IVCML\AppData\Local\Temp\ipykernel_19776\777259552.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  non_final_next_states = Variable(torch.cat(next_states),
C:\Users\IVCML\AppData\Local\Temp\ipykernel_19776\777259552.py:38: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  next_state_values[non_final_mask.view(-1,1)] = model(non_final_next_states).max(1)[0]
C:\Users\IVCML\AppData\Local\Temp\ipykernel_19776\777259552.py:44: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  next_state_values.volatile = False
c:\Users\IVCML\Anaconda3\envs\nactive\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\py

epoch: 0, image: 28, step: 2, reward: -1
epoch: 0, image: 28, step: 3, reward: -1
epoch: 0, image: 28, step: 4, reward: -1
epoch: 0, image: 28, step: 5, reward: -1
epoch: 0, image: 28, step: 6, reward: -1
epoch: 0, image: 28, step: 7, reward: -1
epoch: 0, image: 28, step: 8, reward: -1
epoch: 0, image: 28, step: 9, reward: -1
epoch: 0, image: 29, step: 0, reward: 3
epoch: 0, image: 30, step: 0, reward: -3
epoch: 0, image: 31, step: 0, reward: 1
epoch: 0, image: 31, step: 1, reward: -1
epoch: 0, image: 31, step: 2, reward: -1
epoch: 0, image: 31, step: 3, reward: 1
epoch: 0, image: 31, step: 4, reward: -1
epoch: 0, image: 31, step: 5, reward: -1
epoch: 0, image: 31, step: 6, reward: -3
epoch: 0, image: 32, step: 0, reward: 1
epoch: 0, image: 32, step: 1, reward: 1
epoch: 0, image: 32, step: 2, reward: -1
epoch: 0, image: 32, step: 3, reward: -1
epoch: 0, image: 32, step: 4, reward: -1
epoch: 0, image: 32, step: 5, reward: -1
epoch: 0, image: 32, step: 6, reward: -1
epoch: 0, image: 32, 

# Save the Q-Model

In [15]:
# save the whole model
Q_NETWORK_PATH = '../models/' + 'one_object_model_2'
torch.save(model, Q_NETWORK_PATH)
print('Complete')

Complete
